# This file plots the followings
- End of training mean accuracy in each condition by each h-param setting
- Growth model asymotote, growth (log(k+1)), and x-intercept in the same arrangement

In [ ]:
%load_ext lab_black
import pandas as pd
import numpy as np
import altair as alt
from altair.expr import datum

alt.data_transformers.enable("default")
alt.data_transformers.disable_max_rows()

# Actual plot (EoT)

In [ ]:
df = pd.read_csv("../sims/1250_sims.csv", index_col=0)
df = df.loc[(df["Trial.Scaled"] == 1) & (df.Measure == "Accuracy"), :]
df.sample(5)

In [ ]:
base = (
    alt.Chart(df)
    .mark_rect()
    .encode(
        x="Pnoise:O",
        y="Hidden:O",
        row="Epsilon:O",
        column="PhoHid:O",
        color=alt.Color(
            "mean(Score):Q", scale=alt.Scale(scheme="redyellowgreen", domain=(0.5, 1))
        ),
    )
)

chart = alt.vconcat()
conds = df.Type.unique()

for y in (0, 1, 2):
    row = alt.hconcat()
    for x in (0, 1):
        idx = y * 2 + x
        row |= base.transform_filter(datum.Type == conds[idx]).properties(
            title=conds[idx]
        )
    chart &= row

chart.save("acc_eot_all.html")

# Growth model plots

In [ ]:
df = pd.read_csv("req1_results.csv", index_col=0)
df["learning_rate"] = df.learning_rate.round(3)
df["growth"] = np.log(df.k + 1)
df.sample(5)

### Asymtotes

In [ ]:
large_header = alt.Header(labelFontSize=16, titleFontSize=16)

base = (
    alt.Chart(df)
    .mark_rect()
    .encode(
        x=alt.X("p_noise:O"),
        y=alt.Y("hidden_units:O", sort="descending"),
        column=alt.Column("learning_rate:O", sort="descending", header=large_header,),
        row=alt.Row("cleanup_units", sort="descending", header=large_header,),
        color=alt.Color(
            "mean(max_acc):Q", scale=alt.Scale(scheme="redyellowgreen", domain=(0.5, 1))
        ),
    )
)

In [ ]:
chart = alt.vconcat()
conds = df.cond.unique()

for y in (0, 1, 2):
    row = alt.hconcat()
    for x in (0, 1):
        idx = y * 2 + x
        row |= base.transform_filter(datum.cond == conds[idx]).properties(
            title=conds[idx]
        )
    chart &= row

chart.save("asy_all.html")

### Growth

In [ ]:
sel_df = df.loc[
    df.cond.isin(["HF_INC", "HF_CON", "LF_INC", "LF_CON"]),
]

base = (
    alt.Chart(sel_df)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y=alt.Y("hidden_units:O", sort="descending"),
        row=alt.Row("cleanup_units", sort="descending", header=large_header,),
        column=alt.Column("learning_rate:O", sort="descending", header=large_header,),
        color=alt.Color(
            "mean(growth):Q", scale=alt.Scale(scheme="redyellowblue", domain=(1, 4))
        ),
    )
).configure_axis(labelFontSize=16, titleFontSize=16)


base.save("growth_word_mean.html")

In [ ]:
base = (
    alt.Chart(df)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        row="learning_rate:O",
        column="cleanup_units:O",
        color=alt.Color(
            "mean(growth):Q", scale=alt.Scale(scheme="redyellowblue", domain=(1, 4))
        ),
    )
)

chart = alt.vconcat()
conds = df.cond.unique()

for y in (0, 1, 2):
    row = alt.hconcat()
    for x in (0, 1):
        idx = y * 2 + x
        row |= base.transform_filter(datum.cond == conds[idx]).properties(
            title=conds[idx]
        )
    chart &= row

chart.save("growth_all.html")

### X-Intercept

In [ ]:
base = (
    alt.Chart(df)
    .mark_rect()
    .encode(
        x="p_noise:O",
        y="hidden_units:O",
        row="learning_rate:O",
        column="cleanup_units:O",
        color=alt.Color(
            "mean(x0):Q", scale=alt.Scale(scheme="lightmulti", domain=(0, 0.4))
        ),
    )
)

chart = alt.vconcat()
conds = df.cond.unique()

for y in (0, 1, 2):
    row = alt.hconcat()
    for x in (0, 1):
        idx = y * 2 + x
        row |= base.transform_filter(datum.cond == conds[idx]).properties(
            title=conds[idx]
        )
    chart &= row

chart.save("intercept_all.html")